In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import tensorflow as tf
import numpy as np

import numpy as np


import h5py
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates


In [0]:
def get_onehot_encoder(labels):
    values = array(labels).reshape(-1, 1)
    # integer-binary encode
    onehot_encoder = OneHotEncoder(sparse=False, dtype=np.uint8, categories='auto')
    onehot_encoder.fit(values)
    return onehot_encoder

def get_onehot(labels, onehot_encoder):
    values = array(labels).reshape(-1, 1)
    onehot_encoded = onehot_encoder.transform(values)
    return onehot_encoded
    
def get_onehot_inverse(onehot_label, onehot_encoder):
    # invert first example
    if len(onehot_label.shape) == 1:
        onehot_label = onehot_label.reshape(1,-1)
    values = array(onehot_label)
    inverted = onehot_encoder.inverse_transform(onehot_label)
    return inverted

In [0]:
class Animals:
  def __init__(self, train_images, val_images, test_images, train_labels, val_labels, test_labels):
    self.train_images = train_images
    self.val_images = val_images
    self.test_images = test_images
    self.train_labels = train_labels
    self.val_labels = val_labels
    self.test_labels = test_labels
    self.current_train_batch_index = 0
    self.current_val_batch_index = 0
    self.current_test_batch_index = 0

    self.onehot_encoder = get_onehot_encoder(train_labels)
    self.train_labels_onehot_encoded= get_onehot(train_labels, self.onehot_encoder)
    self.test_labels_onehot_encoded= get_onehot(test_labels, self.onehot_encoder)

  def train_size(self):
    return len(self.train_images)

  def val_size(self):
    return len(self.val_images)

  def test_size(self):
    return len(self.test_images)

  def train_examples(self, examples_size):
    train_examples = self.train_images[:examples_size]
    reshaped_examples = []
    for x in train_examples:
      x = x.reshape(-1)
      reshaped_examples.append(x)
    return np.array(reshaped_examples)


  def next_train_batch(self, batch_size):
    train_images = self.train_images[self.current_train_batch_index: self.current_train_batch_index + batch_size]
    #train_labels = self.train_labels[self.current_train_batch_index: self.current_train_batch_index + batch_size]
    train_labels = self.train_labels_onehot_encoded[self.current_train_batch_index: self.current_train_batch_index + batch_size]
    self.current_train_batch_index += batch_size
    reshaped_batch = []
    for x in train_images:
      x = x.reshape(-1) 
      reshaped_batch.append(x)

    return np.array(reshaped_batch), train_labels

  def next_val_batch(self, batch_size):
    val_images = self.val_images[self.current_val_batch_index: self.current_val_batch_index + batch_size]
    #val_labels = self.val_labels[self.current_val_batch_index: self.current_val_batch_index + batch_size]
    val_labels = self.val_labels_onehot_encoded[self.current_val_batch_index: self.current_val_batch_index + batch_size]
    self.current_val_batch_index += batch_size
    reshaped_batch = []
    for x in val_images:
      x = x.reshape(-1) 
      reshaped_batch.append(x)

    return np.array(reshaped_batch), val_labels

  def next_test_batch(self, batch_size):
    test_images = self.test_images[self.current_test_batch_index: self.current_test_batch_index + batch_size]
    #test_labels = self.test_labels[self.current_test_batch_index: self.current_test_batch_index + batch_size]
    test_labels = self.test_labels_onehot_encoded[self.current_test_batch_index: self.current_test_batch_index + batch_size]
    self.current_test_batch_index += batch_size
    reshaped_batch = []
    for x in test_images:
      x = x.reshape(-1) 
      reshaped_batch.append(x)

    return np.array(reshaped_batch), test_labels

  def reset_batches(self):
    self.current_train_batch_index = 0
    self.current_val_batch_index = 0
    self.current_test_batch_index = 0

  def reset_train_batch(self):
    self.current_train_batch_index = 0

  def reset_test_batch(self):
    self.current_test_batch_index = 0

  def reset_val_batch(self):
    self.current_val_batch_index = 0

In [0]:
def batch_elastic_transform_mnist(images, sigma, alpha, height, width, random_state=None):
    assert len(images.shape) == 2
    # the two lines below ensure we do not alter the array images
    e_images = np.empty_like(images)
    e_images[:] = images
    
    e_images = e_images.reshape(-1, height, width)
    
    if random_state is None:
        random_state = np.random.RandomState(None)
    x, y = np.mgrid[0:height, 0:width]
    
    for i in range(e_images.shape[0]):
        print('hola prro', i)
        dx = gaussian_filter((random_state.rand(height, width) * 2 - 1), sigma, mode='constant') * alpha
        dy = gaussian_filter((random_state.rand(height, width) * 2 - 1), sigma, mode='constant') * alpha
        indices = x + dx, y + dy
        e_images[i] = map_coordinates(e_images[i], indices, order=1)

    return e_images.reshape(-1, 784)

def batch_elastic_transform(images, sigma, alpha, height, width, random_state=None):
    assert len(images.shape) == 2
    # the two lines below ensure we do not alter the array images
    e_images = np.empty_like(images)
    e_images[:] = images
    
    e_images = e_images.reshape(-1, height, width)
    
    if random_state is None:
        random_state = np.random.RandomState(None)
    x, y = np.mgrid[0:height, 0:width]
    
    for i in range(e_images.shape[0]):
        print('hola prro animal', i)
        dx = gaussian_filter((random_state.rand(height, width) * 2 - 1), sigma, mode='constant') * alpha
        dy = gaussian_filter((random_state.rand(height, width) * 2 - 1), sigma, mode='constant') * alpha
        indices = x + dx, y + dy
        e_images[i] = map_coordinates(e_images[i], indices, order=1)

    return e_images.reshape(-1, 196608)

In [0]:
hdf5_path = '/content/drive/My Drive/Colab Notebooks/animals/animals1.h5'
hdf5_file = h5py.File(hdf5_path, "r")
train_images = hdf5_file["train_img"]
val_images = hdf5_file["val_img"]
test_images = hdf5_file["test_img"]

train_labels = hdf5_file["train_labels"]
val_labels = hdf5_file["val_labels"]
test_labels = hdf5_file["test_labels"]

# Import MNIST data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

animals= Animals(train_images, val_images, test_images, train_labels, val_labels, test_labels)
example = animals.train_examples(10)
print(example.shape)

mnist_images, __ = mnist.train.next_batch(1)
#animals_images, _ = animals.next_train_batch(1)

animals_images = animals_images.reshape(256, 256, -1)
#mnist_images = mnist_images.reshape(-1, 28, 28)

#print('Animals', animals_images.shape)

#print('MNIST', mnist_images.shape)

sigma = 4
alpha = 20
  
#(batch_elastic_transform_mnist(mnist_images, sigma, alpha, 28, 28))
#(batch_elastic_transform(animals_images, sigma, alpha, 256, 256))

hdf5_file.close()

(10, 196608)
(256, 256, 30)
